# 使用resnet50解决10-monkeys数据集分类

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os,sys
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
%matplotlib inline

In [5]:
import json
os.chdir('drive/MyDrive/Colab Notebooks')
!mkdir -p ~/.kaggle
!cp ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content/
!kaggle datasets download -d slothkong/10-monkey-species

- path is now set to: /content/
100% 545M/547M [00:08<00:00, 73.0MB/s]
100% 547M/547M [00:08<00:00, 64.8MB/s]


In [6]:
%cd /content

/content


In [7]:
!unzip -o -d /content datasets/slothkong/10-monkey-species/10-monkey-species.zip

Archive:  datasets/slothkong/10-monkey-species/10-monkey-species.zip
  inflating: /content/monkey_labels.txt  
  inflating: /content/training/training/n0/n0018.jpg  
  inflating: /content/training/training/n0/n0019.jpg  
  inflating: /content/training/training/n0/n0020.jpg  
  inflating: /content/training/training/n0/n0021.jpg  
  inflating: /content/training/training/n0/n0022.jpg  
  inflating: /content/training/training/n0/n0023.jpg  
  inflating: /content/training/training/n0/n0024.jpg  
  inflating: /content/training/training/n0/n0025.jpg  
  inflating: /content/training/training/n0/n0026.jpg  
  inflating: /content/training/training/n0/n0027.jpg  
  inflating: /content/training/training/n0/n0028.jpg  
  inflating: /content/training/training/n0/n0029.jpg  
  inflating: /content/training/training/n0/n0030.jpg  
  inflating: /content/training/training/n0/n0031.jpg  
  inflating: /content/training/training/n0/n0032.jpg  
  inflating: /content/training/training/n0/n0033.jpg  
  inflati

In [8]:
train_dir = 'training/training/'
val_dir = 'validation/validation/'
label_file = 'monkey_labels.txt'
print(os.path.exists(train_dir))
print(os.path.exists(val_dir))
print(os.path.exists(label_file))

True
True
True


In [9]:
labels = pd.read_csv(label_file,header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [10]:
height, width = 224,224
channels, batch_size = 1, 24  # 因为图像的尺寸变大了，所以把batch_size较少
num_classes = 10
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    rotation_range = 40, # 把图像岁间旋转0-40度之间
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True, # 水平翻转
) 

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(height,width),
                                                    batch_size=batch_size,seed=8,shuffle=True,
                                                    class_mode='categorical')

# 验证集只需要把像素缩小到0-1之间
val_datagen = keras.preprocessing.image.ImageDataGenerator( 
    preprocessing_function = keras.applications.resnet50.preprocess_input)
val_generator = val_datagen.flow_from_directory(val_dir,target_size=(height,width),
                                               batch_size=batch_size,seed=8,shuffle=True,
                                               class_mode='categorical')
train = train_generator.samples
val = val_generator.samples
print(train,val)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [18]:
resnet50_model = keras.models.Sequential()
resnet50_model.add(keras.applications.ResNet50(include_top=False,pooling='avg',weights='imagenet'))
resnet50_model.add(layers.Dense(num_classes,activation='softmax')) # 增加全连接层，最后输出的类别是10
resnet50_model.layers[0].trainable=False # 把前面的resnet50训练好的参数进行冻结，训练时就只需要训练我们添加的层了，大大的加快了训练速度
resnet50_model.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])


In [19]:
resnet50_model.fit_generator(train_generator,epochs=10,
                             steps_per_epoch=train // batch_size,
                             validation_data = val_generator,
                             validation_steps = val //batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/10
45/45 [==============================] - 50s 1s/step - loss: 1.0355 - accuracy: 0.6918 - val_loss: 0.2828 - val_accuracy: 0.9242
Epoch 2/10
45/45 [==============================] - 46s 1s/step - loss: 0.2064 - accuracy: 0.9590 - val_loss: 0.1275 - val_accuracy: 0.9773
Epoch 3/10
45/45 [==============================] - 41s 920ms/step - loss: 0.1328 - accuracy: 0.9730 - val_loss: 0.1163 - val_accuracy: 0.9697
Epoch 4/10
45/45 [==============================] - 42s 929ms/step - loss: 0.0883 - accuracy: 0.9870 - val_loss: 0.0968 - val_accuracy: 0.9735
Epoch 5/10
45/45 [==============================] - 42s 916ms/step - loss: 0.0714 - accuracy: 0.9870 - val_loss: 0.0804 - val_accuracy: 0.9848
Epoch 6/10
45/45 [==============================] - 41s 921ms/step - loss: 0.0634 - accuracy: 0.9870 - val_loss: 0.0668 - val_accuracy: 0.9735
Epoch 7/10
45/45 [==============================] - 41s 921ms/step - loss: 0.0563 - accuracy: 0.9907 - val_loss: 0.0697 - val_accuracy: 0.9697
Epoch